**TODO:**
* Stemming for all ingredients in recipes
* Remove adjectives from ingredients in recipes
    if this fails, use standardized ingredients from some website
* Create techniques vector, write down 20 techniques and see if they are present in steps string
* 

In [2]:
import pandas as pd
import numpy as np
import ast
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import normalize

In [3]:
def filter_recipes(recipes, ingredients_set):
    """
    removes recipe if ingredients in recipe is not in our ingredients list
    """
    filtered_recipes = []
    for i, recipe in enumerate(recipes):
        found = True
        for word in recipe:
            if word not in ingredients_set:
                found = False
                break
        if found and len(recipe) > 0:
            filtered_recipes.append(recipe)
    return filtered_recipes

In [24]:
recipes_path = 'data/RAW_recipes.csv'
recipes = pd.read_csv(recipes_path)
recipes = recipes['ingredients']
recipes = list(recipes.apply(ast.literal_eval))


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  15

In [6]:
min_count = 1000
model = Word2Vec(min_count=min_count,
                     window=3,
                     size=300,
                     sg=0
                    )
model.build_vocab(recipes, progress_per=100)
vocab = list(model.wv.vocab)
vocab_set = set(vocab)
filtered_recipes = filter_recipes(recipes, vocab_set)

model.train(filtered_recipes, total_examples=model.corpus_count, epochs=30, report_delay=1)

(2305425, 4081560)

In [15]:
model.predict_output_word(['rice'], topn=10)#CBOW

[('diced tomatoes', 0.056856476),
 ('coconut milk', 0.055269446),
 ('beef broth', 0.053055275),
 ('chicken broth', 0.03208546),
 ('coconut', 0.031015953),
 ('cream of mushroom soup', 0.026610376),
 ('tomato sauce', 0.023355428),
 ('kidney beans', 0.020420251),
 ('salt & pepper', 0.017962264),
 ('bay leaves', 0.017291682)]

In [41]:
#TSNE on ingredients embeddings
vocab = list(model.wv.vocab)
word_embeddings = model[vocab]

tsne = TSNE(n_components=2)
word_embeddings_pca = tsne.fit_transform(word_embeddings)

df = pd.DataFrame(word_embeddings_pca, index=vocab, columns=['x', 'y'])
df.head()

fig = plt.figure(figsize=(100,100))
ax = fig.add_subplot(1,1,1)
ax.scatter(df['x'], df['y'])

for ingr, pos in df.iterrows():
    ax.annotate(ingr, pos, )
plt.savefig('ingr_scatter_tsne.png')
plt.close(fig)

/Users/Korvlax/miniconda3/envs/kaggleenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
